# Введение в NumPy

<a href="https://colab.research.google.com/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/02.00-Introduction-to-NumPy.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>

In [1]:
import numpy
numpy.__version__

'1.22.1'

In [2]:
import numpy as np

## Встроенная документация

IPython дает вам возможность быстро изучить содержимое модуля, а также документацию к различным функциям (с помощью символа `?`). Для получения более подробной информации о них обратитесь к [Help and Documentation in IPython](01.01-Help-And-Documentation.ipynb).

Например, чтобы отобразить все содержимое пространства имен NumPy, вы можете набрать следующее:

```ipython
В [3]: np.<TAB>
```

А чтобы отобразить встроенную документацию NumPy, вы можете использовать следующее:

```ipython
In [4]: np?
```

Более подробную документацию, а также учебники и другие ресурсы можно найти на сайте http://www.numpy.org.

# Понимание типов данных в языке Python

Эффективные вычисления, основанные на данных, требуют понимания того, как хранятся данные и как ими манипулировать.
дадим описание и сравним то, как массивы данных обрабатываются в самом языке Python, и как NumPy улучшает это.
д
Пользователей Python часто привлекает простота использования, одной из составляющих которой является динамическая типизация.
В то время как статически типизированные языки, такие как C или Java, требуют явного объявления каждой переменной, динамически типизированные языки, такие как Python, пропускают эту спецификацию. Например, в языке C вы можете указать конкретную операцию следующим образом:

```C
/* Код на C */
int result = 0;
for(int i=0; i<100; i++){
    result += i;
}
```

В то время как в Python эквивалентная операция может быть записана следующим образом:

```python
# Python код
результат = 0
for i in range(100):
    result += i
```

Обратите внимание на главное отличие: в C типы данных каждой переменной объявляются явно, а в Python типы определяются динамически. Это означает, что, например, мы можем присвоить любой тип данных любой переменной:

```python
# Python код
x = 4
x = "четыре"
```

Здесь мы поменяли содержимое ``x`` с целого числа на строку. То же самое в C привело бы (в зависимости от настроек компилятора) к ошибке компиляции или другим непредвиденным последствиям:

```C
/* Код на C */
int x = 4;
x = "четыре"; // ОШИБКА
```

Подобная гибкость - одна из составляющих, которая делает Python и другие динамически типизированные языки удобными и простыми в использовании.
Понимание того, *как* это работает, является важной частью обучения эффективному анализу данных с помощью Python.
Но гибкость типов также указывает на то, что переменные Python - это не просто их значение; они также содержат дополнительную информацию о типе значения.

## Целое число в Python - это больше, чем просто целое число

Стандартная реализация Python написана на языке C.
Это означает, что каждый объект Python - это просто умело замаскированная структура языка C, которая содержит не только свое значение, но и другую информацию. Например, когда мы определяем целое число в Python, например, ``x = 10000``, ``x`` - это не просто "сырое" целое число. На самом деле это указатель на составную структуру C, которая содержит несколько значений.
Просматривая исходный код Python 3.4, мы обнаруживаем, что определение типа integer (long) фактически выглядит следующим образом (после расширения макросов C):

```C
struct _longobject {
    long ob_refcnt;
    PyTypeObject *ob_type;
    size_t ob_size;
    long ob_digit[1];
};
```

Одно целое число в Python 3.4 на самом деле содержит четыре части:

- ``ob_refcnt``, счетчик ссылок, который помогает Python тихо обрабатывать выделение и удаление памяти
- ``ob_type``, который кодирует тип переменной
- ``ob_size``, который определяет размер следующих членов данных
- ``ob_digit``, который содержит фактическое целочисленное значение, которое, как мы ожидаем, будет представлять переменная Python.

Это означает, что хранение целого числа в Python требует определенных затрат по сравнению с целым числом в компилируемом языке, таком как C

![Integer Memory Layout](images/cint_vs_pyint.png)

Здесь ``PyObject_HEAD`` - это часть структуры, содержащая счетчик ссылок, код типа и другие части, упомянутые ранее.

Обратите внимание на разницу: целое число в языке Си - это, по сути, метка для позиции в памяти, байты которой кодируют целочисленное значение.
Целое число в Python - это указатель на позицию в памяти, содержащий всю информацию об объекте Python, включая байты, которые содержат целочисленное значение.
Эта дополнительная информация в целочисленной структуре Python - то, что позволяет кодировать Python так свободно и динамично.
Однако за всю эту дополнительную информацию в типах Python приходится платить, что становится особенно очевидным в структурах, объединяющих множество таких объектов.

## Список Python - это больше, чем просто список

Давайте теперь рассмотрим, что происходит, когда мы используем структуру данных Python, в которой хранится много объектов Python.
Стандартным многоэлементным контейнером с возможностью изменения в Python является список.
Мы можем создать список целых чисел следующим образом:

In [3]:
L = list(range(10))
L

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [4]:
type(L[0])

int

In [5]:
L2 = [str(c) for c in L]
L2

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [6]:
type(L2[0])

str

In [7]:
L3 = [True, "2", 3.0, 4]
[type(item) for item in L3]

[bool, str, float, int]

Но за эту гибкость приходится платить: чтобы обеспечить гибкость типов, каждый элемент списка должен содержать информацию о своем типе, количество ссылок и другую информацию - то есть, каждый элемент является полным объектом Python.
В особом случае, когда все переменные имеют один тип, большая часть этой информации является избыточной: гораздо эффективнее хранить данные в массиве фиксированного типа.

![Array Memory Layout](images/array_vs_list.png)

На уровне реализации массив, по сути, содержит один указатель на один непрерывный блок данных.
Список Python, с другой стороны, содержит указатель на блок указателей, каждый из которых, в свою очередь, указывает на полный объект Python, такой как целое число Python, которое мы рассматривали ранее.
Опять же, преимущество списка заключается в гибкости: поскольку каждый элемент списка представляет собой полную структуру, содержащую как данные, так и информацию о типе, список может быть заполнен данными любого желаемого типа.
Массивы фиксированного типа в стиле NumPy лишены такой гибкости, но гораздо более эффективны для хранения и манипулирования данными.

In [9]:
import numpy as np

## Создание массива из списка Python

In [14]:
# Целочисленный массив
np.array([1, 4, 2, 5, 3])
# Массив чисел с плавающей точкой
np.array([3.14, 4, 2, 3])
np.array([1, 2, 3, 4], dtype='float32')
# Многомерный массив из вложенных списков
np.array([range(i, i + 3) for i in [2, 4, 6]])

array([[2, 3, 4],
       [4, 5, 6],
       [6, 7, 8]])

## Создание массивов с нуля

In [16]:
# Создание массива данных, заполненого нулями
np.zeros(10, dtype=int)
# Создание массива 3x5, заполенного единицами
np.ones((3, 5), dtype=float)
# Создание массива 3x5, заполенного 3.14
np.full((3, 5), 3.14)
# Create an array filled with a linear sequence
# Starting at 0, ending at 20, stepping by 2
# (this is similar to the built-in range() function)
np.arange(0, 20, 2)
# Create an array of five values evenly spaced between 0 and 1
np.linspace(0, 1, 5)
# Create a 3x3 array of uniformly distributed
# random values between 0 and 1
np.random.random((3, 3))
# Create a 3x3 array of normally distributed random values
# with mean 0 and standard deviation 1
np.random.normal(0, 1, (3, 3))
# Create a 3x3 array of random integers in the interval [0, 10)
np.random.randint(0, 10, (3, 3))
# Create a 3x3 identity matrix
np.eye(3)
# Create an uninitialized array of three integers
# The values will be whatever happens to already exist at that memory location
np.empty(3)

array([[3.14, 3.14, 3.14, 3.14, 3.14],
       [3.14, 3.14, 3.14, 3.14, 3.14],
       [3.14, 3.14, 3.14, 3.14, 3.14]])

### NumPy стандартные типы данных

Стандартные типы данных NumPy перечислены в следующей таблице.
Обратите внимание, что при построении массива их можно указать с помощью строки:

```python
np.zeros(10, dtype='int16')
```

Или с помощью связанного объекта NumPy:

```python
np.zeros(10, dtype=np.int16)
```

| Data type	 | Description |
|-------------|-------------|
| `bool_`     | Boolean (True or False) stored as a byte |
| `int_`      | Default integer type (same as C `long`; normally either `int64` or `int32`)| 
| `intc`      | Identical to C `int` (normally `int32` or `int64`)| 
| `intp`      | Integer used for indexing (same as C `ssize_t`; normally either `int32` or `int64`)| 
| `int8`      | Byte (–128 to 127)| 
| `int16`     | Integer (–32768 to 32767)|
| `int32`     | Integer (–2147483648 to 2147483647)|
| `int64`     | Integer (–9223372036854775808 to 9223372036854775807)| 
| `uint8`     | Unsigned integer (0 to 255)| 
| `uint16`    | Unsigned integer (0 to 65535)| 
| `uint32`    | Unsigned integer (0 to 4294967295)| 
| `uint64`    | Unsigned integer (0 to 18446744073709551615)| 
| `float_`    | Shorthand for `float64`| 
| `float16`   | Half-precision float: sign bit, 5 bits exponent, 10 bits mantissa| 
| `float32`   | Single-precision float: sign bit, 8 bits exponent, 23 bits mantissa| 
| `float64`   | Double-precision float: sign bit, 11 bits exponent, 52 bits mantissa| 
| `complex_`  | Shorthand for `complex128`| 
| `complex64` | Complex number, represented by two 32-bit floats| 
| `complex128`| Complex number, represented by two 64-bit floats| 

# Основы работы с массивами NumPy

Работа с данными в Python практически синонимична работе с массивами NumPy: даже такие новые инструменты, как Pandas, построены вокруг массива NumPy.

Здесь мы рассмотрим несколько категорий основных манипуляций с массивами:

- *Атрибуты массивов*: Определение размера, формы, объема памяти и типов данных массивов.
- *Индексация массивов*: Получение и установка значения отдельных элементов массива
- *Разбиение массивов*: Получение и установка меньших подмассивов в большом массиве.
- *Переформирование массивов*: Изменение формы заданного массива
- *Соединение и разделение массивов*: объединение нескольких массивов в один и разбиение одного массива на множество.

## Аттрибуты массивов NumPy

In [18]:
x1 = np.random.randint(10, size=6        )  # Одномерный массив
x2 = np.random.randint(10, size=(3, 4)   )  # Двухмерных массив
x3 = np.random.randint(10, size=(3, 4, 5))  # Трехмерный массив

Каждый массив имеет атрибуты ``ndim`` (количество измерений), ``shape`` (размер каждого измерения) и ``size`` (общий размер массива):

In [19]:
print("x3 ndim: ", x3.ndim )
print("x3 shape:", x3.shape)
print("x3 size: ", x3.size )

x3 ndim:  3
x3 shape: (3, 4, 5)
x3 size:  60


Другим полезным атрибутом является ``dtype``, тип данных массива

In [20]:
print("dtype:", x3.dtype)

dtype: int64


Другие атрибуты включают ``itemsize``, где указан размер (в байтах) каждого элемента массива, и ``nbytes``, где указан общий размер (в байтах) массива.

In [21]:
print("itemsize:", x3.itemsize, "bytes")
print("nbytes:"  , x3.nbytes  , "bytes")

itemsize: 8 bytes
nbytes: 480 bytes


## Индексирование массивов: доступ к отдельным элементам

In [22]:
x1

array([1, 4, 9, 9, 1, 6])

In [23]:
x1[0]

1

In [24]:
x1[-2]

1

In [25]:
x2

array([[1, 5, 7, 2],
       [8, 3, 2, 9],
       [3, 2, 0, 5]])

In [26]:
x2[0, 0]

1

In [27]:
x2[0, -2] = 12
x2

array([[ 1,  5, 12,  2],
       [ 8,  3,  2,  9],
       [ 3,  2,  0,  5]])

In [31]:
x1[-2] = 3.14159  # будет усечено!
x1

array([3, 4, 3, 9, 3, 6])

## Сечение массива: доступ к подмассивам

Можно квадратные скобки использовать для доступа к подмассивам с помощью нотации *slice*, обозначенной символом двоеточия (``:``).
Синтаксис сечения NumPy повторяет стандартный список Python; чтобы получить доступ к подмассива массива ``x`` используется следующий синтаксис:
```python
x[start:stop:step]
```
Если какой-либо из этих параметров не указан, то по умолчанию используются значения ``start=0``, ``stop=``*``размер размерности``*, ``step=1``.
Мы рассмотрим доступ к подмассивам в одном измерении и в нескольких измерениях.

### Одномерный подмассив

In [32]:
x = np.arange(10)
x

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [38]:
x[:5]  
x[5:]
x[4:7]
x[::2]
x[5::-2]

array([5, 3, 1])

### Многомерный подмассив

Многомерные сечения работают аналогичным образом, при этом несколько сечений разделяются запятыми.

In [39]:
x2

array([[ 1,  5, 12,  2],
       [ 8,  3,  2,  9],
       [ 3,  2,  0,  5]])

In [40]:
x2[:2, :3]

array([[ 1,  5, 12],
       [ 8,  3,  2]])

In [41]:
x2[::-1, ::-1]

array([[ 5,  0,  2,  3],
       [ 9,  2,  3,  8],
       [ 2, 12,  5,  1]])

#### Доступ к строкам и столбцам массива

Одной из часто встречающихся процедур является доступ к отдельным строкам или столбцам массива.

In [42]:
print(x2[:, 0])

[1 8 3]


In [43]:
print(x2[0, :])

[ 1  5 12  2]


In [44]:
print(x2[0]) 

[ 1  5 12  2]


### Подмассивы как представления без копирования

Одна важная и чрезвычайно полезная вещь, которую нужно знать о сечении массивов, заключается в том, что они возвращают *представления (view)*, а не *копии (copy)* данных массива.
Это одна из областей, в которой нарезка массивов NumPy отличается от нарезки списков Python: в списках нарезки будут копиями.

In [46]:
print(x2)

[[ 1  5 12  2]
 [ 8  3  2  9]
 [ 3  2  0  5]]


In [47]:
x2_sub = x2[:2, :2]
print(x2_sub)

[[1 5]
 [8 3]]


In [48]:
x2_sub[0, 0] = 99
print(x2_sub)

[[99  5]
 [ 8  3]]


In [50]:
print(x2)

[[99  5 12  2]
 [ 8  3  2  9]
 [ 3  2  0  5]]


Это поведение по умолчанию на самом деле довольно полезно: при работе с большими наборами данных мы можем получать доступ и обрабатывать их части без необходимости копировать основной буфер данных.

### Создание копий массивов

Несмотря на приятные возможности представления массивов, иногда полезно явно копировать данные внутри массива или подмассива. Это проще всего сделать с помощью метода ``copy()``.

In [51]:
x2_sub_copy = x2[:2, :2].copy()
print(x2_sub_copy)

[[99  5]
 [ 8  3]]


In [52]:
x2_sub_copy[0, 0] = 42
print(x2_sub_copy)

[[42  5]
 [ 8  3]]


In [53]:
print(x2)

[[99  5 12  2]
 [ 8  3  2  9]
 [ 3  2  0  5]]


## Изменение формы (reshaping) массивов

Еще один полезный тип операций - изменение формы массивов.
Наиболее гибко это можно сделать с помощью метода ``reshape``.
Например, если вы хотите поместить числа от 1 до 9 в сетку $3 \times 3$, вы можете сделать следующее:

In [54]:
grid = np.arange(1, 10).reshape((3, 3))
print(grid)

[[1 2 3]
 [4 5 6]
 [7 8 9]]


Размер исходного массива должен соответствовать размеру массива c измененной формой. 
По возможности, метод ``reshape`` будет использовать представление исходного массива без копирования, но в случае с несмежными буферами памяти это не всегда так.

Другим распространенным шаблоном изменение формы является преобразование одномерного массива в двумерную матрицу строк или столбцов.
Это можно сделать с помощью метода ``reshape`` или проще - используя ключевое слово ``newaxis`` в операции сечения.

In [55]:
x = np.array([1, 2, 3])

# row vector via reshape
x.reshape((1, 3))

array([[1, 2, 3]])

In [56]:
x[np.newaxis, :]

array([[1, 2, 3]])

In [57]:
x.reshape((3, 1))

array([[1],
       [2],
       [3]])

In [58]:
x[:, np.newaxis]

array([[1],
       [2],
       [3]])

## Конкатенация и разбиение массивов

Все предыдущие процедуры работали с одиночными массивами. 
Можно также объединить несколько массивов в один и, наоборот, разделить один массив на несколько массивов.

### Конкатенация массивов

Конкатенация, или объединение двух массивов в NumPy, в основном выполняется с помощью процедур ``np.concatenate``, ``np.vstack`` и ``np.hstack``.

In [61]:
x = np.array([ 1,  2,  3])
y = np.array([ 3,  2,  1])
z =          [99, 99, 99]
print(np.concatenate([x, y, z]))

[ 1  2  3  3  2  1 99 99 99]


In [63]:
grid = np.array([[1, 2, 3],[4, 5, 6]])

In [65]:
np.concatenate([grid, grid], axis=0)

array([[1, 2, 3],
       [4, 5, 6],
       [1, 2, 3],
       [4, 5, 6]])

In [64]:
np.concatenate([grid, grid], axis=1)

array([[1, 2, 3, 1, 2, 3],
       [4, 5, 6, 4, 5, 6]])

Для работы с массивами смешанных размеров может быть более понятным использование функций ``np.vstack`` вдоль первой оси и ``np.hstack`` вдоль второй оси, ``np.dstack`` вдоль третьей оси..

In [66]:
x_1 = np.array([1, 2, 3])
x_2 = np.array([[9, 8, 7],[6, 5, 4]])

# vertically stack the arrays
np.vstack([x_1, x_2])

array([[1, 2, 3],
       [9, 8, 7],
       [6, 5, 4]])

In [68]:
x_3 = np.array([[99],[99]])
np.hstack([x_2, x_3])

array([[ 9,  8,  7, 99],
       [ 6,  5,  4, 99]])

### Разбиение массивов

Противоположностью конкатенации является разбиение, которое реализуется функциями ``np.split``, ``np.hsplit`` и ``np.vsplit``.  Для каждой из них мы можем передать список индексов, задающих точки разбиения.

In [70]:
x0 = [1, 2, 3, 99, 99, 3, 2, 1]
x1, x2, x3 = np.split(x0, [3, 5])
print(x1, x2, x3)

[1 2 3] [99 99] [3 2 1]
